In [2]:
import pandas as pd
import numpy as np

In [11]:
# read in the data
df = pd.read_csv('/home/dustin/Documents/Study/Master2/CILProject22/data_raw/data_train.csv')
dic = {
    'user_id': [str(x).partition("_")[0][1:] for x in df['Id']],
    'item_id': [str(x).partition("_")[2][1:] for x in df['Id']],
    'combined': [(str(x).partition("_")[0][1:],str(x).partition("_")[2][1:]) for x in df['Id']],
    'prediction': df['Prediction'],
}
data = pd.DataFrame(dic)
data[:100]

,user_id,item_id,combined,prediction
0,44,1,"(44, 1)",4
1,61,1,"(61, 1)",3
2,67,1,"(67, 1)",4
3,72,1,"(72, 1)",3
4,86,1,"(86, 1)",5
...,...,...,...,...
95,2706,1,"(2706, 1)",4
96,2820,1,"(2820, 1)",3
97,2883,1,"(2883, 1)",2
98,2939,1,"(2939, 1)",3


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch as nn
import torch.nn.functional as F


class model(pl.LightningModule):
    def __init__(self):
        super(model,self).__init__()
        embedding_dim = 10
        self.phi = nn.Linear(11000,embedding_dim)
        self.phi_IC = nn.Linear(11000,embedding_dim)
        self.phi_UC = nn.Linear(11000,embedding_dim)
        self.lr = 0.01
        self.optimizer = nn.optim.Adam(self.parameters(), lr=self.lr)
        
    def forward(self, x):
        self.phi(x)
        self.phi_IC(x)
        self.phi_UC(x)
        return self.phi(x), self.phi_IC(x), self.phi_UC(x)
    
    def configure_optimizers(self):
        return nn.optim.Adam(self.parameters(), lr=self.lr)
       
    def training_step(self, train_batch, batch_idx):
        x, ratings = train_batch
        
        phi, phi_IC, phi_UC = self.forward(x)
        loss = self.loss(phi, phi_IC, phi_UC, ratings)
        return loss
        
    def validation_step(self, valid_batch, batch_idx):
        x, y = valid_batch
        phi, phi_IC, phi_U = self.forward(x)
        loss = self.loss(phi, phi_IC, phi_U, y)

    def loss(self, phis, phis_IS, phis_UC, ratings): # implement the loss function
        alpha = 0.1
        lam = 0.1
        reg = 1.0
        phis = phis.transpose(0,1)
        phis_IC = phis_IS.transpose(0,1)
        phis_UC = phis_UC.transpose(0,1)
        loss_DS = - nn.mean(F.logsigmoid(nn.mul(phis[0],phis[1]))) #+ negative sampling
        loss_NS = - nn.mean(F.logsigmoid(nn.mul(phis[0],phis_IC[1]))) - nn.mean(F.logsigmoid(nn.mul(phis[0],phis_UC[0])))
        # norm =    only for overfitting
        loss = alpha*(loss_DS + lam*loss_NS) # + reg*norm)
        return loss

In [ ]:
def find_k_neighborhood(users, items, k):
    # find the k-neighborhood of each user and item
    user_neighborhood = nn.tensor([])
    for user in users:
        for i in range(k):
            rows = data[data['user_id'] == user]
            row = rows.sample(1)
            potential_neighbors = data[data['item_id'] == row['item_id']]
            potential_neighbors = potential_neighbors[potential_neighbors['user_id'] != user]
            neighbor = potential_neighbors.sample(1)
    pass

In [ ]:
# Create Model Object
clf = model()
# Create Data Module Object
train_loader = DataLoader(data['combined'], batch_size=16, shuffle=True)
val_loader = DataLoader(data['combined'], batch_size=16, shuffle=True)
# Create Trainer Object
trainer = pl.Trainer(gpus=1,accelerator='dp',max_epochs=5)
trainer.fit(clf,train_dataloader=train_loader,val_dataloader=val_loader)